## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

## Import needed modules

In [52]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

## Calibrate camera using chessboard images

In [53]:
def calibrate_your_cam():

    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    nx = 9
    ny = 6

    objp = np.zeros((ny*nx,3), np.float32)
    objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    images = glob.glob('../camera_cal/calibration*.jpg')

    # Step through the list and search for chessboard corners
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (nx,ny),None)
        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
            # Draw and display the corners
            # img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
            # cv2.imshow('img',img)
            # cv2.waitKey(500)
    cv2.destroyAllWindows()
    
    # Camera calibration
    # Now, we have a mapping from object points to image points (for each image).
    # Those are stored in the lists objpoints, imgpoints respectively.
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    
    return mtx, dist


In [54]:
# Camera matrix "mtx" (mapping matrix) and distortion cooefficient "dist" can be calculated now.
mtx, dist = calibrate_your_cam()

## 1. Apply a distortion correction to raw images.

In [55]:
# Image example
img = cv2.imread('../test_images/straight_lines1.jpg') # BGR
image = cv2.undistort(img, mtx, dist, None, mtx)
# cv2.imwrite('../output_steps/straight_lines1_undist.jpg', image)
# mng = plt.get_current_fig_manager()
# mng.window.showMaximized()

## 2. Use color transforms, gradients, etc., to create a thresholded binary image.

In [56]:
# Define a function that takes an image, gradient orientation,
# and threshold min / max values.
def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a copy and apply the threshold
    binary_output = np.zeros_like(scaled_sobel)
    # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1

    # Return the result
    return binary_output

# Define a function to return the magnitude of the gradient
# for a given sobel kernel size and threshold values
def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1

    # Return the binary image
    return binary_output

# Define a function to threshold an image for a given range and Sobel kernel
def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1

    # Return the binary image
    return binary_output

# Define a function that thresholds the S-channel of HLS
def hls_select(img, thresh=(0, 255), channel = 2):
    # Assumed cv2.imshow function was used
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    s_channel = hls[:,:,channel]
    binary_output = np.zeros_like(s_channel)
    binary_output[(s_channel > thresh[0]) & (s_channel <= thresh[1])] = 1
    return binary_output

# Combine color and gradiant threshold to create the binary image
def color_grad_thr(image):
    # Use gradient 
    ksize = 15
    bin_sobelx = abs_sobel_thresh(image, orient='x', sobel_kernel=ksize, thresh=(20, 100))
    bin_sobely = abs_sobel_thresh(image, orient='y', sobel_kernel=ksize, thresh=(20, 100))
    bin_sobel_mag = mag_thresh(image, sobel_kernel=ksize, mag_thresh=(30, 100))
    bin_sobel_dir = dir_threshold(image, sobel_kernel=ksize, thresh=(0.7, 1.3))
    bin_grad = np.zeros_like(bin_sobel_dir)
    bin_grad[((bin_sobelx == 1) & (bin_sobely == 1)) | ((bin_sobel_mag == 1) & (bin_sobel_dir == 1))] = 1
    
    # Use color transforms
    bin_color = hls_select(image, thresh=(90, 255), channel=2)
    
    # Gradient and color combined
    binary = np.zeros_like(bin_color)
    binary[(bin_grad == 1) | (bin_color == 1)] = 1
    
    return bin_grad, bin_color, binary

In [57]:
bin_grad, bin_color, binary = color_grad_thr(image)
plt.subplot(221)
plt.title('Original undistorted')
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.subplot(222)
plt.title('Gradiant')
plt.imshow(bin_grad, 'gray')
plt.subplot(223)
plt.title('S channel')
plt.imshow(bin_color, 'gray')
plt.subplot(224)
plt.title('Gradient and color combined')
plt.imshow(binary, 'gray')
# cv2.imwrite('../output_steps/binary_combo.jpg', binary.astype('uint8') * 255)
# mng = plt.get_current_fig_manager()
# mng.window.showMaximized()

## 3. Apply a perspective transform to rectify binary image ("birds-eye view").

In [58]:
# Define a function that takes an image, number of x and y points, 
# camera matrix and distortion coefficients
def warper(img):
    # Choose offset from image corners to plot detected corners
    # This should be chosen to present the result at the proper aspect ratio
    # My choice of 100 pixels is not exact, but close enough for our purpose here
    # Grab the image shape
    img_size = (img.shape[1], img.shape[0])
    # print("image.shape[0] =", img.shape[0])
    # print("image.shape[1] =", img.shape[1])
    # src = np.float32([corners[0], corners[nx-1], corners[-1], corners[-nx]])
    p1, p2, p3, p4 = (180, img.shape[0]), (550, 476), (740, 476), (1125, img.shape[0])
    offset = 300
    q1, q2, q3, q4 = (offset, img.shape[0]-1), (offset, 1), (img.shape[1]-offset, 1), (img.shape[1]-offset, img.shape[0]-1)
    src = np.float32([p1, p2, p3, p4])
    # For destination points, I'm arbitrarily choosing some points to be
    # a nice fit for displaying our warped result 
    # again, not exact, but close enough for our purposes
    dst = np.float32([q1, q2, q3, q4])
    # Given src and dst points, calculate the perspective transform matrix
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    # Warp the image using OpenCV warpPerspective()
    warped = cv2.warpPerspective(img, M, img_size)
    
    lineThickness = 2
    if 0:
        cv2.line(image, p1, p2, (0,255,0), lineThickness)
        cv2.line(image, p2, p3, (0,255,0), lineThickness)
        cv2.line(image, p3, p4, (0,255,0), lineThickness)
        cv2.line(image, p4, p1, (0,255,0), lineThickness)
        cv2.line(warped, q1, q2, (0,255,0), lineThickness)
        cv2.line(warped, q2, q3, (0,255,0), lineThickness)
        cv2.line(warped, q3, q4, (0,255,0), lineThickness)
        cv2.line(warped, q4, q1, (0,255,0), lineThickness)

    # Return the resulting image and matrix
    return warped, M, src, dst, Minv

In [59]:
binary_warped, M, src, dst, Minv = warper(binary)

plt.subplot(121)
plt.title('Original test')
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))


plt.subplot(122)
plt.title('BEV')
plt.imshow(binary_warped, 'gray')
# plt.imshow(cv2.cvtColor(binary_warped, cv2.COLOR_BGR2RGB))

mng = plt.get_current_fig_manager()
mng.window.showMaximized()


## 4. Detect lane pixels and fit to find the lane boundary.

In [60]:
def find_lane_pixels_win(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img

def fit_poly_win(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels_win(binary_warped)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0])
    try:
        plotx_left = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        plotx_right = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        plotx_left = 1*ploty**2 + 1*ploty
        plotx_right = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    # Plots the left and right polynomials on the lane lines
    # plt.plot(left_fitx, ploty, color='yellow')
    # plt.plot(right_fitx, ploty, color='yellow')
    # out_img[ploty.astype(int), left_fitx.astype(int)] = [255, 255, 0]
    # cv2.polylines(out_img, np.int32([left_fitx, ploty]), 1, (255, 255, 0))
    # print ([left_fitx, ploty])
    # points = np.dstack([left_fitx, ploty])
    # print (np.int32([points]))
    # cv2.polylines(out_img, np.int32([points]), 0, (255, 255, 0), 3)
    
    
    return out_img, left_fit, right_fit, plotx_left, plotx_right, ploty

def fit_poly(img_shape, leftx, lefty, rightx, righty, real_world=False):
     ### TO-DO: Fit a second order polynomial to each with np.polyfit() ###
    if real_world == False: 
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)
    else:
        left_fit = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
        right_fit = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    ### TO-DO: Calc both polynomials using ploty, left_fit and right_fit ###
    plotx_left = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    plotx_right = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return left_fit, right_fit, plotx_left, plotx_right, ploty

def search_around_poly(binary_warped, left_fit, right_fit, real_world=False):
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    # The quiz grader expects 100 here, but feel free to tune on your own!
    margin = 100

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    ### TO-DO: Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    left_fit, right_fit, plotx_left, plotx_right, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty, real_world)
        
    if real_world == False:
    ## Visualization ##
        # Create an image to draw on and an image to show the selection window
        out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
        window_img = np.zeros_like(out_img)
        # Color in left and right line pixels
        out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
        out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

        # Generate a polygon to illustrate the search window area
        # And recast the x and y points into usable format for cv2.fillPoly()
        left_line_window1 = np.array([np.transpose(np.vstack([plotx_left-margin, ploty]))])
        left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([plotx_left+margin, 
                                  ploty])))])
        left_line_pts = np.hstack((left_line_window1, left_line_window2))
        right_line_window1 = np.array([np.transpose(np.vstack([plotx_right-margin, ploty]))])
        right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([plotx_right+margin, 
                                  ploty])))])
        right_line_pts = np.hstack((right_line_window1, right_line_window2))

        # Draw the lane onto the warped blank image
        cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
        cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
        result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)

        ## End visualization steps ##
        return result, left_fit, right_fit, plotx_left, plotx_right, ploty
    else:
        return left_fit, right_fit, plotx_left, plotx_right, ploty

In [61]:
# Define conversions in x and y from pixels space to meters
ym_per_pix = 30/720 # meters per pixel in y dimension
xm_per_pix = 3.7/700 # meters per pixel in x dimension

out_img, left_fit, right_fit, plotx_left, plotx_right, ploty = fit_poly_win(binary_warped)
plt.imshow(out_img)
plt.plot(plotx_left, ploty, color='yellow')
plt.plot(plotx_right, ploty, color='yellow')

# Polynomial fit values from the previous frame
result, left_fit, right_fit, plotx_left, plotx_right, ploty = search_around_poly(binary_warped, left_fit, right_fit)
plt.imshow(result)
plt.plot(plotx_left, ploty, color='yellow')
plt.plot(plotx_right, ploty, color='yellow')
mng = plt.get_current_fig_manager()
mng.window.showMaximized()

## 5. Determine the curvature of the lane and vehicle position with respect to center.

In [62]:
def measure_curvature_real(left_fit_cr, right_fit_cr, ploty):
    '''
    Calculates the curvature of polynomial functions in meters.
    '''
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    # Start by generating our fake example data
    # Make sure to feed in your real data instead in your project!
    # ploty, left_fit_cr, right_fit_cr = generate_data(ym_per_pix, xm_per_pix)
    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    ##### TO-DO: Implement the calculation of R_curve (radius of curvature) #####
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    return left_curverad, right_curverad

def calc_curv_dist_to_center(binary_warped, left_fit, right_fit):
    
    # Curvature
    left_fit, right_fit, plotx_left, plotx_right, ploty = search_around_poly(binary_warped, left_fit, right_fit, True)
    left_curverad_real, right_curverad_real = measure_curvature_real(left_fit, right_fit, ploty)
    
    # print("left_curverad_real", left_curverad_real)
    # print("right_curverad_real", right_curverad_real)
    # Distance to center
    y_eval = ym_per_pix * np.max(ploty)
    camera_center_x = xm_per_pix * (binary_warped.shape[1]/2)
    # print("binary_warped.shape[1]/2", binary_warped.shape[1]/2)
    # print("camera_center_x", camera_center_x)
    lane_left_x = left_fit[0]*y_eval**2 + left_fit[1]*y_eval + left_fit[2]
    # print("lane_left_x", lane_left_x)
    lane_right_x = right_fit[0]*y_eval**2 + right_fit[1]*y_eval + right_fit[2]
    # print("lane_right_x", lane_right_x)
    dis_to_center_left = camera_center_x - lane_left_x
    dis_to_center_right = lane_right_x - camera_center_x
    return left_curverad_real, right_curverad_real, dis_to_center_left, dis_to_center_right

## 6. Warp the detected lane boundaries back onto the original image.

In [63]:
def warp_back(img, binary_warped, Minv, left_fitx, right_fitx, ploty):

    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(img, 1, newwarp, 0.3, 0)
    
    return result

# plt.subplot(211)

# plt.subplot(212)
# plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [64]:
result = warp_back(img, binary_warped, Minv, plotx_left, plotx_right, ploty)
plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))

## 7. Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

In [65]:
def write_text(image, left_curverad_real, right_curverad_real, dis_to_center_left, dis_to_center_right):
    if dis_to_center_left > dis_to_center_right:
        side = "right"
    else:
        side = "left"
    veh_pos = np.abs(dis_to_center_left - dis_to_center_right)
    str_curv_left = "Radius of left curvature = {:.0f} m".format(left_curverad_real)
    str_curv_right = "Radius of right curvature = {:.0f} m".format(right_curverad_real)
    str_veh_center = "Vehicle is {:.2f} m ".format(veh_pos) + side + " of center"
    cv2.putText(image, str_curv_left, (100, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255))
    cv2.putText(image, str_curv_right, (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255))
    cv2.putText(image, str_veh_center, (100, 150), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255))
    return image
    # plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))

In [66]:
left_curverad_real, right_curverad_real, dis_to_center_left, dis_to_center_right = calc_curv_dist_to_center(binary_warped, left_fit, right_fit)
result = write_text(result, left_curverad_real, right_curverad_real, dis_to_center_left, dis_to_center_right)
plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))

In [67]:
def draw_image(image, title, cmap = "gray"):
    plt.figure()
    plt.title(title)    
    plt.xlabel('x')
    plt.ylabel('y')
    if cmap != "gray":
        image2 = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        plt.imshow(image2)
    else:
        plt.imshow(image, "gray")

In [68]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = []
        #average x values of the fitted line over the last n iterations
        self.bestx = None
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None 

In [69]:
def fill_line_object(line_object, fit_parameters, curverad, dis_to_center):
    n = 5
    line_object.current_fit = fit_parameters
    line_object.radius_of_curvature = curverad
    line_object.line_base_pos = dis_to_center
    # print("len(line_object.recent_xfitted)", len(line_object.recent_xfitted))
    if len(line_object.recent_xfitted) < n:
        line_object.recent_xfitted.append(fit_parameters)
        # print("line_object.recent_xfitted\n", line_object.recent_xfitted)
    else:
        # print("before delete\n", line_object.recent_xfitted)
        line_object.recent_xfitted.remove(line_object.recent_xfitted[0])
        # print("after delete\n", line_object.recent_xfitted)
        line_object.recent_xfitted.append(fit_parameters)
        # print("line_object.recent_xfitted\n", line_object.recent_xfitted)
    
    line_object.bestx = np.average(line_object.recent_xfitted, axis=0)
    # print("line_object.bestx", line_object.bestx)
    

## Pipeline

In [70]:
# Camera calibration matrix and distortion coefficients are calculated above and can be used for each new frame
# mtx, dist are used to calculate undistorted image

def pipeline(img):
    global state, line_left, line_right
    # Undistort input img
    image = cv2.undistort(img, mtx, dist, None, mtx)
    # plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    if 0:
        draw_image(image, "Undistorted image", "RGB")
    # Use color transforms, gradients, etc., to create a thresholded binary image.
    bin_grad, bin_color, binary = color_grad_thr(image)
    if 0:
        draw_image(binary, "Color and gradient binary", "gray")
    # Apply a perspective transform to rectify binary image ("birds-eye view").
    binary_warped, M, src, dst, Minv = warper(binary)
    if 0:
        draw_image(binary_warped, "Warped binary", "gray")
    # Detect lane pixels and fit to find the lane boundary.
    # print("state", state)
    if state == 'reset':
        # Windows
        out_img, line_left.current_fit, line_right.current_fit, plotx_left, plotx_right, ploty  = fit_poly_win(binary_warped)
        line_left.bestx, line_right.bestx = line_left.current_fit, line_right.current_fit
        state = 'ok'
        if 0:
            draw_image(out_img, "Pixels and lane lines (Windows)", "RGB")
            plt.plot(plotx_left, ploty, color='yellow')
            plt.plot(plotx_right, ploty, color='yellow')
    elif state == 'ok':    
        # Gates
        result, line_left.current_fit, line_right.current_fit, plotx_left, plotx_right, ploty = search_around_poly(binary_warped, line_left.bestx, line_right.bestx, real_world=False)
        if 0:
            draw_image(result, "Pixels and lane lines (Gates)", "RGB")
            # Plot the polynomial lines onto the image
            plt.plot(plotx_left, ploty, color='yellow')
            plt.plot(plotx_right, ploty, color='yellow')
    
    # Determine the curvature of the lane and vehicle position with respect to center.
    left_curverad_real, right_curverad_real, dis_to_center_left, dis_to_center_right = calc_curv_dist_to_center(binary_warped, line_left.bestx, line_right.bestx)
    
    # Warp the detected lane boundaries back onto the original image.
    result = warp_back(img, binary_warped, Minv, plotx_left, plotx_right, ploty)
    if 0:
        draw_image(result, "Warped back", "RGB")
    # Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.
    result = write_text(result, left_curverad_real, right_curverad_real, dis_to_center_left, dis_to_center_right)
    if 0:
        draw_image(result, "Warped back with text", "RGB")
    # print("current left_fit", left_fit)
    fill_line_object(line_left, line_left.current_fit, left_curverad_real, dis_to_center_left)
    fill_line_object(line_right, line_right.current_fit, right_curverad_real, dis_to_center_right)
    return result

In [71]:
state = 'reset'
line_left = Line()
line_right = Line()
img = cv2.imread('../test_images/problem.png') # BGR
result = pipeline(img)
output_path = r'../test_images/problem_processed.png'
cv2.imwrite(output_path, result)

True

## Call pipeline on a video input

In [49]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [51]:
line_left = Line()
line_right = Line()
state = 'reset'
# Output
output_file = '../project_video_out.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
# Input
project_video_clip = VideoFileClip("../project_video.mp4")
processed_clip = project_video_clip.fl_image(pipeline) #NOTE: this function expects color images!!
%time processed_clip.write_videofile(output_file, audio=False)

t:   0%|          | 0/1260 [00:00<?, ?it/s, now=None]

Moviepy - Building video ../project_video_out.mp4.
Moviepy - Writing video ../project_video_out.mp4



Moviepy - Done !
Moviepy - video ready ../project_video_out.mp4
CPU times: user 6min 57s, sys: 15.9 s, total: 7min 13s
Wall time: 6min 5s
